In [0]:
dbutils.widgets.text("proc_date", "")
proc_date = dbutils.widgets.get("proc_date")

In [0]:
%run ../../../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
create_temp_view_with_clean_columns(
    spark,
    catalog_name,
    'udp_wcm_bronze_sap_bw',
    'sap_bw_wcm_vccmmrb28',
    proc_date,
    'temp_sap_vccmmrb28'
)

In [0]:
spark.sql(
    f"""
CREATE TABLE IF NOT EXISTS {catalog_name}.udp_wcm_silver_sap_bw.sap_vccmmrb28 (
    kappl STRING,
    kschl STRING,
    vkorg STRING,
    vtweg STRING,
    konda STRING,
    matnr STRING,
    vrkme STRING,
    kfrst STRING,
    datbi DATE,
    recordmode STRING,
    datab DATE,
    kbstat STRING,
    knumh STRING,
    proc_date DATE,
    file_creation_ts TIMESTAMP)
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)"""
)

In [0]:
spark.sql(f"""
INSERT INTO {catalog_name}.udp_wcm_silver_sap_bw.sap_vccmmrb28
SELECT
  kappl,
  kschl,
  vkorg,
  vtweg,
  konda,
  LTRIM('0', matnr) AS matnr,
  vrkme,
  kfrst,
  TO_DATE(datbi, 'yyyyMMdd') AS datbi,
  recordmode,
  TO_DATE(datab, 'yyyyMMdd') AS datab,
  kbstat,
  knumh,
  proc_date,
  file_creation_ts
FROM temp_sap_vccmmrb28;
""")